In [1]:
import torch
from src.model import swin_v2

config_path = "checkpoints/hydro_simmim_pretrain_swinv2_base_img256_window16_800ep.yaml"


In [3]:
model, transforms, config = swin_v2(config_path)

=> merge config from checkpoints/hydro_simmim_pretrain_swinv2_base_img256_window16_800ep.yaml


In [4]:
# Extract image level embeddings
with torch.inference_mode():
    x = torch.randn(1, 12, 256, 256)
    x = transforms(x)
    embeddings = model.forward_features(x)
    print(embeddings.shape)

torch.Size([1, 1024])


In [5]:
# Extract intermediate feature embeddings
with torch.inference_mode():
    x = torch.randn(1, 12, 256, 256)
    x = transforms(x)
    features = model.get_intermediate_layers(x, n=(0, 1, 2, 3, 4), reshape=True)
    for i, f in enumerate(features):
        print(i, f.shape)

0 torch.Size([1, 128, 64, 64])
1 torch.Size([1, 256, 32, 32])
2 torch.Size([1, 512, 16, 16])
3 torch.Size([1, 1024, 8, 8])
4 torch.Size([1, 1024, 8, 8])
